In [ ]:
# hide
%load_ext autoreload
%autoreload 2

In [ ]:
# default_exp services.bbox_trajectory

In [ ]:
# hide
from nbdev import *

In [ ]:
#exporti
from ipycanvas import Canvas
from typing import List
from ipyannotator.mltypes import BboxCoordinate

In [ ]:
#exporti

class BBoxTrajectory:
    @staticmethod
    def draw_trajectory(canvas: Canvas, coords: List[BboxCoordinate], scale: float = 1.0):
        # iterate the coords two by two
        i, k = None, None
        c = iter(coords)
        lines = []
        while True:
            if i is None:
                i = next(c, None)
            k = next(c, None)
            if k and i:
                lines.append([(i.x * scale, (i.y + i.height) * scale),
                              (k.x * scale, (k.y + k.height) * scale)])
            else:
                break
            i = k

        canvas.stroke_styled_line_segments(lines, color=[50, 205, 50])

In [ ]:
# hide
from ipyannotator.bbox_canvas import draw_bounding_box, draw_bg

In [ ]:
#hide

# it can draw point in the middle of the circle

canvas = Canvas(width=100, height=100)
bbox_trajectory = BBoxTrajectory()
bbox_coords = [
    BboxCoordinate(*[0, 0, 50, 50]),
    BboxCoordinate(*[10, 20, 50, 50])
]

draw_bg(canvas)
draw_bounding_box(canvas, bbox_coords[0])
draw_bounding_box(canvas, bbox_coords[1])
bbox_trajectory.draw_trajectory(canvas=canvas, coords=bbox_coords)
canvas

In [ ]:
#hide

from attr import asdict
from ipyannotator.mltypes import BboxVideoCoordinate
from itertools import groupby
from collections import defaultdict

canvas = Canvas(width=500, height=500)
draw_bg(canvas)

storage = {
    'path1': {
        "bboxes": [
            BboxVideoCoordinate(**{'x': 10, 'y': 10, 'width': 103,
                                'height': 241, 'id': 'pedestrian1'}),
            BboxVideoCoordinate(**{'x': 100, 'y': 350, 'width': 100,
                                'height': 100, 'id': 'pedestrian2'}),
            BboxVideoCoordinate(**{'x': 300, 'y': 100, 'width': 155,
                                'height': 156, 'id': 'pedestrian3'})
        ],
        'labels': [[], [], []]
    },
    'path2': {
        'bboxes': [
            BboxVideoCoordinate(**{'x': 30, 'y': 30, 'width': 102,
                                'height': 241, 'id': 'pedestrian1'})
        ],
        'labels': [[]]
    }
}

trajectory = defaultdict(list)


def key_fun(k):
    return k.id


for k, v in storage.items():
    for kk, vv in groupby(sorted(v['bboxes'], key=key_fun), key_fun):
        value = list(vv)
        path = []
        for i in value:
            bbox_coordinate = asdict(i)
            bbox_coordinate.pop('id')
            path.append(BboxCoordinate(**bbox_coordinate))

        trajectory[kk] += path

In [ ]:
#hide

for k, v in trajectory.items():
    for bbox in v:
        draw_bounding_box(canvas, bbox)
    if len(v) > 1:
        bbox_trajectory.draw_trajectory(canvas, v)

canvas

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()